<a href="https://colab.research.google.com/github/sanchit2843/DLExtras/blob/master/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -i https://test.pypi.org/simple/ supportlib

Looking in indexes: https://test.pypi.org/simple/


In [0]:
import supportlib.gettingdata as getdata
getdata.kaggle()

Saving kaggle.json to kaggle.json


In [0]:
!kaggle competitions download -c dog-breed-identification

  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 88.2MB/s]
  0% 0.00/281k [00:00<?, ?B/s]
100% 281k/281k [00:00<00:00, 86.9MB/s]
 97% 335M/346M [00:03<00:00, 99.9MB/s]
100% 346M/346M [00:03<00:00, 103MB/s] 
 99% 342M/345M [00:03<00:00, 121MB/s]
100% 345M/345M [00:03<00:00, 95.3MB/s]


In [0]:
getdata.zipextract('./train.zip')
getdata.zipextract('./labels.csv.zip')

In [0]:
import pandas as pd
data = pd.read_csv('./labels.csv')
#data.head(10)

In [0]:
encoder = {}
decoder = {}
for i,label in enumerate(data['breed'].unique()):
  encoder[label] = i
  decoder[i] = label

In [0]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys
class dataset(Dataset):
    def __init__(self,data,dir,transform = None):
        self.data = data
        self.dir = dir
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        file = self.data['id'][idx]
        path = os.path.join(self.dir,file + '.jpg')
        image = cv2.imread(path)
        if self.transform:
          image = self.transform(image)
        return image,encoder[self.data['breed'][idx]]
      
im_size = 224
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]


train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

train_data = dataset(data,'./train',transform = train_transforms)
train_loader = DataLoader(train_data,batch_size = 64,num_workers = 8,shuffle = True)

In [0]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      #image = image_batch[n]*255.0
      plt.imshow(image)
      plt.title(decoder[label_batch[n].numpy()])
      plt.axis('off')

In [0]:
import time
batch_size = 16
default_timeit_steps = 1000
def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    if(i== len(ds)):
      it = iter(ds)
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()
  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(batch_size*steps/duration))

In [0]:
timeit(train_loader)

....................................................................................................
1000 batches: 83.11957240104675 s
192.49377 Images/s


In [0]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [0]:
class classifie(nn.Module):
    def __init__(self):
        super(classifie, self).__init__()
        model = models.densenet121(pretrained = True)
        model.classifier = Identity()
        self.model = model
        self.linear1 = nn.Linear(1024,120)
    def forward(self, input):
        am = self.model(input)
#        x = self.dropout(self.relu(self.linear1(am)))
        x = self.linear1(am)
        return x

In [0]:
model = classifie().to('cuda')
from torchsummary import summary
summary(model,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
       BatchNorm2d-5           [-1, 64, 56, 56]             128
              ReLU-6           [-1, 64, 56, 56]               0
            Conv2d-7          [-1, 128, 56, 56]           8,192
       BatchNorm2d-8          [-1, 128, 56, 56]             256
              ReLU-9          [-1, 128, 56, 56]               0
           Conv2d-10           [-1, 32, 56, 56]          36,864
      BatchNorm2d-11           [-1, 96, 56, 56]             192
             ReLU-12           [-1, 96, 56, 56]               0
           Conv2d-13          [-1, 128, 56, 56]          12,288
      BatchNorm2d-14          [-1, 128,

In [0]:
device = 'cuda'
cls_criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

In [0]:
from torch.autograd import Variable
iteration = 0
acc_all = list()
loss_all = list()
dataloaders = {'train':train_loader} 
num_epochs = 10
for epoch in range(num_epochs):
    print('')
    print(f"--- Epoch {epoch} ---")
    phase1 = dataloaders.keys()
    for phase in phase1:
        print('')
        print(f"--- Phase {phase} ---")
        epoch_metrics = {"loss": [], "acc": []}
        for batch_i, (X, y) in enumerate(dataloaders[phase]):
            image_sequences = Variable(X.to(device), requires_grad=True)
            labels = Variable(y.to(device), requires_grad=False)
            optimizer.zero_grad()
            predictions = model(image_sequences)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
            loss.backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)
            batches_done = epoch * len(dataloaders[phase]) + batch_i
            batches_left = num_epochs * len(dataloaders[phase]) - batches_done
            sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                    % (
                        epoch,
                        num_epochs,
                        batch_i,
                        len(dataloaders[phase]),
                        loss.item(),
                        np.mean(epoch_metrics["loss"]),
                        acc,
                        np.mean(epoch_metrics["acc"]),
                    )
                )

                # Empty cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
            
        print('')
        print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
        torch.save(model.state_dict(),'/content/model.h5'.format(epoch))
        if(phase=='train'):
          acc_all.append(np.mean(epoch_metrics["acc"]))
          loss_all.append(np.mean(epoch_metrics["loss"]))


--- Epoch 0 ---

--- Phase train ---
[Epoch 0/10] [Batch 638/639] [Loss: 4.806834 (5.048705), Acc: 0.00% (0.91%)]
train , acc: 0.9096244131455399

--- Epoch 1 ---

--- Phase train ---
[Epoch 1/10] [Batch 638/639] [Loss: 4.850374 (4.753492), Acc: 0.00% (1.59%)]
train , acc: 1.5942879499217528

--- Epoch 2 ---

--- Phase train ---
[Epoch 2/10] [Batch 638/639] [Loss: 4.590879 (4.646283), Acc: 0.00% (2.12%)]
train , acc: 2.12245696400626

--- Epoch 3 ---

--- Phase train ---
[Epoch 3/10] [Batch 638/639] [Loss: 4.700273 (4.468488), Acc: 7.14% (3.39%)]
train , acc: 3.3855913257321704

--- Epoch 4 ---

--- Phase train ---
[Epoch 4/10] [Batch 330/639] [Loss: 4.224683 (4.359656), Acc: 6.25% (4.72%)]

In [0]:
for param in model.parameters():
  param.requires_grad = True
train_loader = DataLoader(train_data,batch_size = 16,num_workers = 4,shuffle = True)

In [0]:
from torch.autograd import Variable
iteration = 0
acc_all = list()
loss_all = list()
dataloaders = {'train':train_loader} 
num_epochs = 10
for epoch in range(num_epochs):
    print('')
    print(f"--- Epoch {epoch} ---")
    phase1 = dataloaders.keys()
    for phase in phase1:
        print('')
        print(f"--- Phase {phase} ---")
        epoch_metrics = {"loss": [], "acc": []}
        for batch_i, (X, y) in enumerate(dataloaders[phase]):
            image_sequences = Variable(X.to(device), requires_grad=True)
            labels = Variable(y.to(device), requires_grad=False)
            optimizer.zero_grad()
            predictions = model(image_sequences)
            loss = cls_criterion(predictions, labels)
            acc = 100 * (predictions.detach().argmax(1) == labels).cpu().numpy().mean()
            loss.backward()
            optimizer.step()
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)
            batches_done = epoch * len(dataloaders[phase]) + batch_i
            batches_left = num_epochs * len(dataloaders[phase]) - batches_done
            sys.stdout.write(
                    "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)]"
                    % (
                        epoch,
                        num_epochs,
                        batch_i,
                        len(dataloaders[phase]),
                        loss.item(),
                        np.mean(epoch_metrics["loss"]),
                        acc,
                        np.mean(epoch_metrics["acc"]),
                    )
                )

                # Empty cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
        print('')
        print('{} , acc: {}'.format(phase,np.mean(epoch_metrics["acc"])))
        torch.save(model.state_dict(),'/content/model.h5'.format(epoch))
        if(phase=='train'):
          acc_all.append(np.mean(epoch_metrics["acc"]))
          loss_all.append(np.mean(epoch_metrics["loss"]))


--- Epoch 0 ---

--- Phase train ---
[Epoch 0/10] [Batch 638/639] [Loss: 4.847332 (4.993830), Acc: 0.00% (1.04%)]
train , acc: 1.0367762128325508

--- Epoch 1 ---

--- Phase train ---
[Epoch 1/10] [Batch 255/639] [Loss: 4.842217 (4.836545), Acc: 0.00% (0.85%)]

KeyboardInterrupt: ignored